# Assignment 1: Transformations


TEAM-ID:17
<br>
TEAM-NAME: no-flux-given  
YOUR-ID:   
YOUR-NAME:    

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
from time import sleep
import math

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [ ]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################
# Replace "pass" statement with your code
def one_one():
    color = o3d.io.read_image("color.jpg")
    depth = o3d.io.read_image("depth.png")
    rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(color, depth)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
    
    world_pcd = mesh.sample_points_uniformly(number_of_points=10000)
    
    scene_pcd = world_pcd + pcd
    
    o3d.io.write_point_cloud("scene.pcd", scene_pcd)
    o3d.visualization.draw_geometries([scene_pcd])
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [ ]:
#uncomment the following and add input parameters if any
one_one()

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################
# Replace "pass" statement with your code
def two_one():
    side = 1.0
    cube = o3d.geometry.TriangleMesh.create_box(side, side, side)
    pcd = cube.sample_points_uniformly(number_of_points=10000000)
    center = [side/2, side/2, side/2]

    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    
    picked = vis.get_picked_points()
    points = np.asarray(pcd.points)
    if len(picked) == 0:
        link = np.zeros((3, 1))
    
    elif len(picked) >= 1:
        picked = picked[0]
        link = np.array(points[picked])
        link = np.array(link).reshape(3, 1)

    cube_new = cube.translate(link)
    pcd_new = cube_new.sample_points_uniformly(number_of_points=10000000)

    center = np.array(center).reshape(3,1)
    new_center = link + center
    new_center = np.array(new_center).reshape(3, 1)
    cube_new.compute_vertex_normals()
    cube_paint = cube_new.paint_uniform_color([0.1, 0.1, 0.7])

    coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=side+0.5)
    coordinate = coordinate.translate(center)
    coordinate = coordinate.translate(link)
    coordinate.compute_vertex_normals()

    mesh = cube_paint + coordinate

    mesh_pcd = mesh.sample_points_uniformly(number_of_points=10000000)
    o3d.io.write_point_cloud("cube.pcd", mesh_pcd)

    scene = o3d.io.read_point_cloud("scene.pcd")
    combined = scene + mesh_pcd

    source = mesh_pcd
    new = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0)
    new = new.translate(center)
    new = new.translate(link)

    new = new.sample_points_uniformly(number_of_points=10000000)
    target =  new + scene
    vis = o3d.visualization.Visualizer()

    angle_z = 30
    angle_y = 90
    angle_x = 45
    vis.create_window()
    vis.add_geometry(source)
    vis.add_geometry(target)

    angle = 0
    while angle <= angle_z:
        if(angle ==0):
            i = 0
        else:
            i = 5
        r = R.from_euler('ZYX', [i, 0, 0], degrees=True)
        source.rotate(np.array(r.as_matrix()), center=new_center)
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 5
        sleep(0.01)

    r = R.from_euler('ZYX', [30, 0, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    y_vector = rotation @ (np.array([0, 1, 0]).reshape(3, 1))

    angle = 5
    while angle <= angle_y:
        if(angle ==0):
            i = 0
        else:
            i = 5 
        new = y_vector * np.pi*5/180
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=new_center)
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 5
        sleep(0.1)

    r = R.from_euler('ZYX', [30, 90, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    x_vector = rotation @ (np.array([1, 0, 0]).reshape(3, 1))

    angle = 5
    while angle <= angle_x:
        if(angle ==0):
            i = 0
        else:
            i = 5
        new = x_vector * np.pi*5/180
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=new_center)
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 5
        sleep(0.01)

    sleep(4.0)
    vis.destroy_window()
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################


### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [ ]:
#uncomment the following and add input parameters if any
two_one()

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [5]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [6]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code
def euler2rot(alpha, beta, gamma):
    
    z = np.array([[math.cos(alpha), -math.sin(alpha), 0], 
                   [math.sin(alpha), math.cos(alpha), 0],
                   [0, 0, 1]])
    y = np.array([[math.cos(beta), 0, math.sin(beta)], 
                   [0, 1, 0],
                   [-math.sin(beta), 0, math.cos(beta)]])
    x = np.array([[1, 0, 0], 
                   [0, math.cos(gamma), -math.sin(gamma)],
                   [0, math.sin(gamma), math.cos(gamma)]])
    recovered = np.dot(z, np.dot(y, x))
    return recovered

In [9]:
#Case 1 

m = math.sqrt(M_given[0,0]**2 + M_given[1,0]**2)

alpha = math.atan2(M_given[1,0],M_given[0,0]) 
beta = math.atan2(-M_given[2,0],m)
gamma = math.atan2(M_given[2,1],M_given[2,2])
eulerm = [alpha,beta,gamma]

M_recovered = euler2rot(alpha, beta, gamma)

print("Calculations using scipy:")
r = R.from_matrix(M_given)
e = r.as_euler('ZYX')
print("Euler Angles from Rotation matrix: " + str(e))

r = R.from_euler('ZYX', e)
print("Rotation matrix from Euler Angles: \n" + str(r.as_matrix()))

Calculations using scipy:
Euler Angles from Rotation matrix: [0.69813171 1.22173048 0.52359878]
Rotation matrix from Euler Angles: 
[[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]


In [10]:
#Case 2

beta = math.radians(90); #since cos(beta)=0 and sin(beta)=1
alpha = 0; #assumption
gamma = math.atan2(N_given[0,1], N_given[1,1])
eulern = np.array([alpha, beta, gamma])

N_recovered = euler2rot(alpha, beta, gamma)

print("Calculations using scipy:")
r = R.from_matrix(N_given)
e = r.as_euler('ZYX')
print("Euler Angles from Rotation matrix: " + str(e))

r = R.from_euler('ZYX', e)
print("Rotation matrix from Euler Angles: \n" + str(r.as_matrix()))

Calculations using scipy:
Euler Angles from Rotation matrix: [0.17453293 1.57079633 0.        ]
Rotation matrix from Euler Angles: 
[[ 0.00000000e+00 -1.73648178e-01  9.84807753e-01]
 [ 0.00000000e+00  9.84807753e-01  1.73648178e-01]
 [-1.00000000e+00  0.00000000e+00 -5.55111512e-17]]


C:\Users\sagri\anaconda3\envs\mr_assignment1\lib\site-packages\scipy\spatial\transform\rotation.py:133: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  warnings.warn("Gimbal lock detected. Setting third angle to zero since"


### Code for 2.2 Optional Section 

In [11]:
#Case 1

r1 = R.from_matrix(M_given)
q1 = r1.as_quat()
print("Quaternion for Case 1: " + str(q1))

#Case 2

r2 = R.from_matrix(N_given)
q2 = r2.as_quat()
print("Quaternion for Case 2: " + str(q2))

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Quaternion for Case 1: [0.00973605 0.59313249 0.13112033 0.79429624]
Quaternion for Case 2: [-0.06162842  0.70441603  0.06162842  0.70441603]


## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: We have used math.atan2(y,x) which returns arctan(y/x) since the relationship between euler angles and the elements of rotation matrix are defined in such a manner. Atan2(y,x) computes tan−1(y/x) but uses the signs of both x and y to identify the quadrant in which the resulting angle lies. A single argument function like 'atan' won't be able to identify that correctly.

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [12]:
# Uncomment and replace my_array_case1 with your array.
print("My Euler angles for case 1 are" + str(eulerm))

My Euler angles for case 1 are[0.6981317003487097, 1.2217304765297095, 0.5235987753730347]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [13]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans:     There is no problem with euler angle representation in case 1. Therefore, we were easily able to recover the rotation matrix from euler angles. The components of rotation matrix are just direction cosines with euler angles as arguments. The rotation matrix defined by euler angles is as follows: 
    
$\begin{bmatrix}
c\alpha c\beta & c\alpha s\beta s\gamma - s\alpha c\gamma & c\alpha s\beta c\gamma + s\alpha s\gamma\\
s\alpha c\beta & s\alpha s\beta s\gamma + c\alpha c\gamma & s\alpha s\beta c\gamma - c\alpha s\gamma\\
-s\beta & c\beta s\gamma & c\beta c\gamma\\
\end{bmatrix}$

### Repeat the above for Case 2.

In [14]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are" + str(eulern))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are[ 0.          1.57079633 -0.17453293]
For case 2, it is True I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: Yes there is a problem in this case because multiple solutions for euler angles exist and this situation is called gimbal lock.    
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: At a gimbal lock, axes of two angular rotations are aligned which results in infinite solutions for euler angles. It can also be said that the representation loses a degree of freedom in a gimbal lock.

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: We used 'ZYX' in r.as_ruler('') as it is case sensitive. Upper case gives intrinsic rotation and lower case gives extrinsic rotation. Set of intrinsic rotations is also called euler angles and set of extrinsic rotations is called fixed angles. Extrinsic: Each of the three rotations takes place about an axis in the fixed reference frame. Intrinsic: Each rotation is performed about an axis of the moving system rather than one of the fixed reference.
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: It shows a warning in the case of gimbal lock (Case 2) where there can be infinite solutions for euler angles. Hence, scipy gives only one solution where third angle (gamma) is set to zero.
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: The scipy function for quaternion didn't give any warning in Case 2 implying that gimbal lock is avoided. Quaternion might be better than euler angles in such cases.


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \textbf{Ans: }(cos\alpha \times sin\beta \times sin\gamma - sin\alpha \times cos\gamma) - 1/\sqrt{3} = 0 \\ (sin\alpha \times sin\beta \times sin\gamma + cos\alpha \times cos\gamma) - 1/\sqrt{3} = 0 \\
   cos\beta \times sin\gamma - 1/\sqrt{3} = 0 $$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = -0.39780642, -93.75690197, 0.71372205 (in radians)* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = 0.00969376, 0.79019829, 0.96218762 (in radians)*

In [7]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [15]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################
# Replace "pass" statement with your code

def func(x):
    return [math.cos(x[0])*math.sin(x[1])*math.sin(x[2]) - math.sin(x[0])*math.cos(x[2]) - 1/math.sqrt(3), 
            math.sin(x[0])*math.sin(x[1])*math.sin(x[2]) + math.cos(x[0])*math.cos(x[2]) - 1/math.sqrt(3),
            math.cos(x[1])*math.sin(x[2]) - 1/math.sqrt(3)]

#Case 1
init = [0,0,0]
root = fsolve(func, init)
[a1, b1, c1] = root


#Case 2
five = math.radians(5)
init = [0-five, np.pi/4-five,  np.arccos(math.sqrt(3)/3)-five]
root2 = fsolve(func, init)
[a2, b2, c2] = root2

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

In [16]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = euler2rot(a1,b1,c1) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = euler2rot(a2,b2,c2) @ x_1.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [17]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - When $\beta$ is $90^\circ$ and $-90^\circ$, we enter a gimbal lock and the two axes x and z get aligned which is why we are never able to reach the point $(x,y,z)$
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [19]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code
def two_four():
    #Case 1 alpha = -35, beta = 90
    side = 1.0
    coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=side)
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)

    angle_z = -35
    angle_y = 90
    angle_x = 55
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    source = coordinate
    target = sphere
    vis.add_geometry(source)
    vis.add_geometry(target)

    angle = 0
    while angle >= angle_z:
        if(angle ==0):
            i = 0
        else:
            i = -5
        r = R.from_euler('ZYX', [i, 0, 0], degrees=True)
        source.rotate(np.array(r.as_matrix()), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle -= 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, 0, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    y_vector = rotation @ (np.array([0, 1, 0]).reshape(3, 1))

    angle = 5
    while angle <= angle_y:
        if(angle ==0):
            i = 0
        else:
            i = 5 
        new = y_vector * np.pi*5/180
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, angle_y, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    x_vector = rotation @ (np.array([1, 0, 0]).reshape(3, 1))

    angle = -180
    while angle <= 180:
        new = x_vector * (np.pi*10/180)
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 10
        sleep(0.5)

    vis.destroy_window()
    
    #Case 2 alpha = 45, beta = 90
    side = 1.0
    coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=side)
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)

    angle_z = 45
    angle_y = 90
    angle_x = 55
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    source = coordinate
    target = sphere
    vis.add_geometry(source)
    vis.add_geometry(target)

    angle = 0
    while angle <= angle_z:
        if(angle ==0):
            i = 0
        else:
            i = 5
        r = R.from_euler('ZYX', [i, 0, 0], degrees=True)
        source.rotate(np.array(r.as_matrix()), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, 0, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    y_vector = rotation @ (np.array([0, 1, 0]).reshape(3, 1))

    angle = 5
    while angle <= angle_y:
        if(angle ==0):
            i = 0
        else:
            i = 5 
        new = y_vector * np.pi*5/180
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, angle_y, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    x_vector = rotation @ (np.array([1, 0, 0]).reshape(3, 1))

    angle = -180
    while angle <= 180:
        new = x_vector * (np.pi*10/180)
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 10
        sleep(0.5)

    vis.destroy_window()
    
    #Case 3 alpha = -35, beta = -90
    side = 1.0
    coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=side)
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)

    angle_z = -35
    angle_y = -90
    angle_x = 55
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    source = coordinate
    target = sphere
    vis.add_geometry(source)
    vis.add_geometry(target)

    angle = 0
    while angle >= angle_z:
        if(angle ==0):
            i = 0
        else:
            i = -5
        r = R.from_euler('ZYX', [i, 0, 0], degrees=True)
        source.rotate(np.array(r.as_matrix()), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle -= 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, 0, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    y_vector = rotation @ (np.array([0, 1, 0]).reshape(3, 1))

    angle = -5
    while angle >= angle_y:
        new = y_vector * np.pi*(-5)/180
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle -= 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, angle_y, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    x_vector = rotation @ (np.array([1, 0, 0]).reshape(3, 1))

    angle = -180
    while angle <= 180:
        new = x_vector * (np.pi*10/180)
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 10
        sleep(0.5)

    vis.destroy_window()
    
    #Case 4 alpha = 45, beta = -90
    side = 1.0
    coordinate = o3d.geometry.TriangleMesh.create_coordinate_frame(size=side)
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)

    angle_z = 45
    angle_y = -90
    angle_x = 55
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    source = coordinate
    target = sphere
    vis.add_geometry(source)
    vis.add_geometry(target)

    angle = 0
    while angle <= angle_z:
        if(angle ==0):
            i = 0
        else:
            i = 5
        r = R.from_euler('ZYX', [i, 0, 0], degrees=True)
        source.rotate(np.array(r.as_matrix()), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, 0, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    y_vector = rotation @ (np.array([0, 1, 0]).reshape(3, 1))

    angle = -5
    while angle >= angle_y:
        new = y_vector * np.pi*(-5)/180
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle -= 5
        sleep(0.5)

    r = R.from_euler('ZYX', [angle_z, angle_y, 0], degrees=True)
    rotation = np.array(r.as_matrix())
    x_vector = rotation @ (np.array([1, 0, 0]).reshape(3, 1))

    angle = -180
    while angle <= 180:
        new = x_vector * (np.pi*10/180)
        source.rotate(source.get_rotation_matrix_from_axis_angle(new), center=(0, 0, 0))
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        angle += 10
        sleep(0.5)

    vis.destroy_window()
    
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: ${90^\circ}$ and ${-90^\circ}$
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans:  The rotation matrix in this case ($\beta=-90^\circ$)  will be:
    $$
    \begin{bmatrix}
    0 & -c\alpha s\gamma - s\alpha c\gamma & -c\alpha c\gamma + s\alpha s\gamma\\
    0 & -s\alpha s\gamma + c\alpha c\gamma & -s\alpha c\gamma - c\alpha s\gamma\\
    1 & 0 & 0\\
    \end{bmatrix}
    =  \begin{bmatrix}
        0 & -sin(\alpha + \gamma) & -cos(\alpha + \gamma)\\
        0 & cos(\alpha + \gamma) & -sin(\alpha + \gamma)\\
        1 & 0 & 0\\
        \end{bmatrix}
    $$
    
    As you can see, there are only two degrees of freedom in this case, $\beta$ and ($\alpha+\gamma$). Therefore we can say that we have lost a degree of freedom which implies that the set of rotations has resulted in a gimbal lock.
    Similarly for $\beta=-90^\circ$, only two degrees of freedom $\beta$ and ($\alpha-\gamma$) will exist which is again a case of gimbal lock.
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [5]:
#uncomment the following and add input parameters if any
two_four()